# Testing Queries

Note: For now, we are lucky that the imports work, because we are using relative imports, and at one point, it will cause errors.

## IMPORTS

In [1]:
import os
import sys
import random
import requests
import argparse
from typing import List, Set

import numpy as np
import pandas as pd

In [2]:
from db.yagodb import YagoDB
from db.constants.main import YAGO_ALL_ENTITY_COUNT, YAGO_FACTS_ENTITY_COUNT
from db.functions.entity import get_random_entities_query

In [3]:
from kg.query import get_triples_multiple_subjects_query, get_description_multiple_entities_query, query_kg, get_triples_from_response

In [4]:
from utils.constants import YAGO_ENTITY_STORE_DB_PATH, YAGO_PREFIXES_PATH, YAGO_ENDPOINT_URL
from utils.prefix import get_prefixes, get_url_from_prefix_and_id
from utils.random_walk2 import SPARQL_COLUMNS_DICT, RandomWalk2

### CONSTANTS

### FUNCTIONS

### Experiment Single Walks

In [5]:
yago_db = YagoDB(YAGO_ENTITY_STORE_DB_PATH)

In [6]:
random_walk_2 = RandomWalk2(yago_db)

In [7]:
entities_df = random_walk_2.random_walk_description_batch(num_of_entities=10, depth=3)

In [8]:
entities_df

,entity0,description0,predicate1,entity1,description1,predicate2,entity2,description2
0,http://yago-knowledge.org/resource/_U005B_Fbc2...,galaxy in the constellation Carina,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Galaxy,large gravitationally bound system of stars an...,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Deep-sky_ob...,any astronomical object that is not an individ...
1,http://yago-knowledge.org/resource/_U0028_2R_U...,chemical compound,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Chemical_co...,pure chemical substance consisting of two or m...,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Pure_Substa...,chemical substance composed of atoms or molecu...
2,http://yago-knowledge.org/resource/Louis_Frolla,Monegasque writer (1904-1978),http://schema.org/birthPlace,http://yago-knowledge.org/resource/Monaco,sovereign city-state on the French Riviera,http://yago-knowledge.org/resource/neighbors,http://yago-knowledge.org/resource/European_Union,political and economic union of 27 European st...
3,http://yago-knowledge.org/resource/Leda_37546_...,galaxy in the constellation Leo,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Galaxy,large gravitationally bound system of stars an...,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Deep-sky_ob...,any astronomical object that is not an individ...
4,http://yago-knowledge.org/resource/Martin_Lütt...,NaN,None,None,NaN,None,None,NaN
5,http://yago-knowledge.org/resource/Ucac3_66-68...,star in the constellation Carina,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Star,astronomical object consisting of a luminous s...,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Fusor__u002...,astronomical object that achieves core fusion ...
6,http://yago-knowledge.org/resource/Effect_Of_O...,clinical trial,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Clinical_trial,phase of clinical research in medicine,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Product_tes...,a process of measuring the properties or perfo...
7,http://yago-knowledge.org/resource/Wohnhaus_In...,NaN,None,None,NaN,None,None,NaN
8,http://yago-knowledge.org/resource/No_Matter_W...,1998 single by Boyzone,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Single__u00...,group of single releases by an artist usually ...,http://schema.org/alternateName,singel,NaN
9,http://yago-knowledge.org/resource/Q20143943,human settlement in Mexico,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/AdministrativeArea,"community of any size, in which people live",http://schema.org/alternateName,bestuurlijke indeling,NaN


In [11]:
entities_df["description0"].isna().sum()

4

In [10]:
entities_df.iloc[9]["entity0"]

'http://yago-knowledge.org/resource/Canton_De_Bagnols-Sur-Cèze_Q88077655'

In [10]:
entities_df.head(10).to_csv("walks.csv", index=False)